In [1]:
import os
import csv
from PIL import Image
from typing import Tuple
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from torchvision import models, transforms

In [5]:
# Step 2: Data Collection and Labeling
# Class for loading and labeling inputs
class ArchitecturalDataset(Dataset):
    def __init__(self, images_dir: str, labels_csv: str, transform=None):
        self.images_dir = images_dir
        self.transform = transform
        self.data = []
        
        # Load the CSV and store entries
        with open(labels_csv, 'r', newline='', encoding='utf-8') as f:
            reader = csv.reader(f)
            for row in reader:
                filename, compliance, thickness = row
                self.data.append({
                    "filename": filename,
                    "compliance": compliance,
                    "thickness": float(thickness)
                })
        
        # Map compliance strings to numeric classes
        # Let's say: 0 = non-compliant, 1 = compliant
        self.label_map = {"non-compliant": 0, "compliant": 1}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx: int):
        entry = self.data[idx]
        img_path = os.path.join(self.images_dir, entry["filename"])
        
        # Load image
        image = Image.open(img_path).convert("RGB")
        
        # Get label
        label = self.label_map[entry["compliance"]]

        if self.transform:
            image = self.transform(image)
        

        return image, label

In [5]:
# Step 3: Data Preprocessing
# Define the transformations for training and validation
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)), # model required image size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]) # input normalization
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Directory structure reminder
# data/
#   train/
#       detail_01.jpg
#       detail_02.jpg
#       ...
#   val/
#       detail_04.jpg
#       detail_05.jpg
#       ...
#   train_labels.csv
#   val_labels.csv

train_dataset = ArchitecturalDataset(images_dir="/Users/haobo2022/Documents/ML-Design-Review/data/train",
                                     labels_csv="/Users/haobo2022/Documents/ML-Design-Review/data/train_labels.csv",
                                     transform=train_transforms)

val_dataset = ArchitecturalDataset(images_dir="/Users/haobo2022/Documents/ML-Design-Review/data/val",
                                   labels_csv="/Users/haobo2022/Documents/ML-Design-Review/data/val_labels.csv",
                                   transform=val_transforms)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=2)

FileNotFoundError: [Errno 2] No such file or directory: 'data/train_labels.csv'

In [ ]:
# Step 4: Model Architecture
# Load a pretrained ResNet model (ResNet18 for simplicity, could try ResNet 50 in the future to capture more nuance)
model = models.resnet18(pretrained=True)

# Modify the final fully connected (fc) layer for 2 classes: [non-compliant, compliant]
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

# At this point, model outputs the following classifications.
# Class 0: non-compliant
# Class 1: compliant

# Output logic (include after model training)
# If prediction == 0:
#    recommendation = "Recommendation: Increase insulation thickness to at least 5 inches as per code Y to ensure proper thermal resistance."
# Else:
#    recommendation = "No violation detected. The current design meets the minimum insulation requirements."

In [ ]:
# Model training and interations
num_epochs = 5  # Start with something small
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        running_corrects += torch.sum(preds == labels).item()
        total_train += labels.size(0)

    train_loss = running_loss / total_train
    train_acc = running_corrects / total_train

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_corrects = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            val_corrects += torch.sum(preds == labels).item()
            total_val += labels.size(0)

    val_loss = val_loss / total_val
    val_acc = val_corrects / total_val

    print(f"Epoch {epoch+1}/{num_epochs} - "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} - "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")